In [0]:
pip freeze | grep soda | xargs pip uninstall -y

Found existing installation: isodate 0.6.1
Not uninstalling isodate at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-5b464307-594e-462d-89d9-bfbd0d87f02e
Can't uninstall 'isodate'. No files were found to uninstall.
Found existing installation: soda 1.0.1
Uninstalling soda-1.0.1:
  Successfully uninstalled soda-1.0.1
Found existing installation: soda-freshness 0.0.3
Uninstalling soda-freshness-0.0.3:
  Successfully uninstalled soda-freshness-0.0.3
Found existing installation: soda-spark 1.0.1
Uninstalling soda-spark-1.0.1:
  Successfully uninstalled soda-spark-1.0.1
Found existing installation: soda-spark-df 1.0.1
Uninstalling soda-spark-df-1.0.1:
  Successfully uninstalled soda-spark-df-1.0.1


In [0]:
pip install -i https://pypi.cloud.soda.io soda-spark-df

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Looking in indexes: https://pypi.cloud.soda.io
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 29.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
# import Scan from Soda Library
from soda.scan import Scan
# Create a Spark DataFrame, or use the Spark API to read data and create a DataFrame
df = spark.table("delta.`/databricks-datasets/nyctaxi/tables/nyctaxi_yellow`")
# Create a view that SodaCL uses as a dataset
df.createOrReplaceTempView("NYC_Taxi_Data")

# Create a Scan object, set a scan definition, and attach a Spark session
scan = Scan()
scan.set_scan_definition_name("Databricks Notebook")
scan.set_data_source_name("spark_df")
scan.add_spark_session(spark)

# Define checks for datasets
# TODO: Extend checks for demo purposes
checks = """
checks for NYC_Taxi_Data:
  - row_count > 0:
      name: Row Count Not Zero
sample datasets:
  datasets:
    - include NYC_Taxi_Data
"""
# If you defined checks in a file accessible via Spark, you can use the scan.add_sodacl_yaml_file method to retrieve the checks
scan.add_sodacl_yaml_str(checks)
# Optionally, add a configuration file with Soda Cloud credentials
config ="""
soda_cloud:
  host: demo.soda.io
  api_key_id: 399b27f5-f51a-4463-9992-3ce241ab53c9
  api_key_secret: hNSg7Fu4NngGVX--95RmdwIkpVpjVGrhr0cvVxpUPSQ4PD77xD_W1Q
"""
scan.add_configuration_yaml_str(config)
# Execute a scan
scan.execute()

# Check the Scan object for methods to inspect the scan result; the following prints all logs to console
print(scan.get_logs_text()) 

# Use the additional scan methods to insert circuit breakers into your processes, e.g. scan.assert_no_checks_fail